In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import feyn

# Model agnostic 
from typing import Optional, List, Callable, Dict, Any, List
from pathlib import Path
from utils import ChiefBldr, QLatticeWrapper

This version of Feyn and the QLattice is available for academic, personal, and non-commercial use. By using the community version of this software you agree to the terms and conditions which can be found at https://abzu.ai/eula.

In [2]:
# Get the directory this file lives in
nb_dir = Path.cwd() # notebook directory
project_root = nb_dir.parents[0] # project directory
data_path = project_root / "datasets" / "processed_well_data.csv"

includ_cols = ['Dia', 'Dev(deg)','Area (m2)', 'z','GasDens','LiquidDens', 'P/T','friction_factor', 'critical_film_thickness']
D = ChiefBldr(path=data_path, includ_cols=includ_cols, test_size=0.20)

In [3]:
# define xgboost pipeline
def qlattice(
        hparams: Dict[str,Any]
):
    ql_wrap = QLatticeWrapper(
        feature_tags=includ_cols, 
        **hparams,
    )

    return ql_wrap

hparam_grid = {
            "max_complexity":   [10, 15, 25],
            "n_epochs":         [10, 15, 20],
        }
# train model and optimize hyperparameters via grid search 
trained_model = D.evolv_model(build_model=qlattice, hparam_grid=hparam_grid, k_folds=5)

# output equation 
print(trained_model.express())

Best cross-validation score and std = 0.6746880570409981 0.06173839958552623
Best hyperparameters: {'max_complexity': 15, 'n_epochs': 15}
Test set class score: 0.6190476190476191


1.92459*tanh(1.05783*Area (m2) + 0.164482*P/T + 0.184731*frictionfactor + 0.590456) - 0.563413
